# **Analysis of Independent Presidential Candidata Dr. Iitula's Manifesto for 2019  Election Campaign**

### **1. Imports**

In [1]:
import PyPDF2
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

### **2. Read our document**

Lets read our pdf for the manifesto using the `PdfFileReader()` function from the PyPDF2 which is a package for extracting document information such as **title, author, number of pages,....**, spliting documents page by page, merging page by page, etc. 

In [2]:
filename = 'manibook'
open_filename = open(filename, 'rb')

ind_manifesto = PyPDF2.PdfFileReader(open_filename)


FileNotFoundError: [Errno 2] No such file or directory: 'Manifesto/manibook'

To get the document informtion  ussing the `getDocumentInfo()` function and check the number of pages in our document using the `numPages()` function. There are various useful functions one can use to check other things. See online documentation:[PyPDF2](https://pythonhosted.org/PyPDF2/index.html)

In [ ]:
ind_manifesto.getDocumentInfo()

In [ ]:
total_pages = ind_manifesto.numPages
total_pages

From the outputs of our two previous codes, we got the **title** of the document, what OS was used to type the document, when the document was created and modified. And we also got the total number of pages in our document. 

### **3. Lets extract the texts from the pdf file and print it**

We will use a `textract` package to extract our texts from the document.

In [ ]:
import textract   

Loop throug all the pages in the document and extract the text from it 

In [ ]:
count = 0
text  = ''

# Lets loop through, to read each page from the pdf file
while(count < total_pages):
    # Get the specified number of pages in the document
    mani_page  = ind_manifesto.getPage(count)
    # Process the next page
    count += 1
    # Extract the text from the page
    text += mani_page.extractText()
   

The `if` statement check if our document returned words from the loop above using the `extractText()` function. This is done since `PyPDF2` cannot read scanned documents. 

In [ ]:
if text != '':
    text = text
    
else:
    textract.process(open_filename, method='tesseract', encoding='utf-8', langauge='eng' )    

If the above returns a false, then run the Optical Character Recognition (OCR) `textract` to convert scanned/image based Pdf files to text. See `textract` online documentaion: [textract](https://textract.readthedocs.io/en/stable/python_package.html).

Lets print out our texts to see what it contains which was converted to lower case using the `lower()` function.

In [ ]:
from autocorrect import Speller
from nltk.tokenize import word_tokenize


def to_lower(text):

    """
    Converting text to lower case as in, converting "Hello" to  "hello" or "HELLO" to "hello".
    """
    
    # Specll check the words
    spell  = Speller(lang='en')
    
    texts = spell(text)
    
    return ' '.join([w.lower() for w in word_tokenize(text)])

lower_case = to_lower(text)
print(lower_case)

And clear from seeing the printed text, we only extracted texts from page 2 till last pace. The reason being is that, those pages that we did not extract text from are in image based and we failed to to do. **SOMEONE CAN PERHAPS HELP!!!!.** 

### **4. Clean our *to_lower_case* text variable and return it as a list of keywords.**

From the printed text, it's apparent that our text contains unwanted characters such as spaces, punctuations `\n` and so forth. 

Lets break our text phrases into individual words using `word_tokenize()` function from the Naturalge Toolkit (nltk).

In [ ]:
import nltk
import re
import string
from nltk.corpus import stopwords, brown
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from autocorrect import spell

In [ ]:
def clean_text(lower_case):
    # split text phrases into words
    words  = nltk.word_tokenize(lower_case)
    
    
    # Create a list of all the punctuations we wish to remove
    punctuations = ['.', ',', '/', '!', '?', ';', ':', '(',')', '[',']', '-', '_', '%']
    
    # Remove all the special characters
    punctuations = re.sub(r'\W', ' ', str(lower_case))
    
    # Initialize the stopwords variable, which is a list of words ('and', 'the', 'i', 'yourself', 'is') that do not hold much values as key words
    stop_words  = stopwords.words('english')
    
    # Getting rid of all the words that contain numbers in them
    w_num = re.sub('\w*\d\w*', '', lower_case).strip()
    
    # remove all single characters
    lower_case = re.sub(r'\s+[a-zA-Z]\s+', ' ', lower_case)
    
    # Substituting multiple spaces with single space
    lower_case = re.sub(r'\s+', ' ', lower_case, flags=re.I)
    
    # Removing prefixed 'b'
    lower_case = re.sub(r'^b\s+', '', lower_case)
    
    
    
    # Removing non-english characters
    lower_case = re.sub(r'^b\s+', '', lower_case)
    
    # Return keywords which are not in stop words 
    keywords = [word for word in words if not word in stop_words  and word in punctuations and  word in w_num]
    
    return keywords

In [ ]:
# Lemmatize the words
wordnet_lemmatizer = WordNetLemmatizer()

lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in clean_text(lower_case)]

# lets print out the output from our function above and see how the data looks like
clean_data = ' '.join(lemmatized_word)
print(clean_data)

In [ ]:
import pandas as pd


Lets save our data into a dataframe so we can do our anal

In [ ]:
df = pd.DataFrame([clean_data])
df.columns = ['script']
df.index = ['Itula']
df

###  **5. Preprocess - Bag of Words model** 

The bag-of-words model is a simplifying representation used in natural language processing and information retrieval (IR). In this model, a text (such as a sentence or a document) is represented as the bag (multiset) of its words, disregarding grammar and even word order but keeping multiplicity. The bag-of-words model has also been used for computer vision [Wikipedia](https://stackabuse.com/python-for-nlp-sentiment-analysis-with-scikit-learn/).


It is mostly used to extract features from text for used in modelling, such as machine learning algorithms.

In [ ]:
#  Counting the occurrences of tokens and building a sparse matrix of documents x tokens.
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

corpus = df.script
vect = CountVectorizer(stop_words='english')

# Transforms the data into a bag of words
data_vect = vect.fit_transform(corpus)

Applt the document-term matrix which is a mathematical matrix which decribes the frequency of words in a document

In [ ]:
feature_names = vect.get_feature_names()
data_vect_feat = pd.DataFrame(data_vect.toarray(), columns=feature_names)
data_vect_feat.index = df.index
data_vect_feat

Our vector representations show the frequency of words used in the document

In [ ]:
data = data_vect_feat.transpose()
data

### **6. Getting the top 100 frequent words from the manifesto.**

We will try to get the top most common 100 words from our document and plot that into a wordcloud for visualization.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn

# Find the top 1000 words written in the manifesto
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False)
    top_dict[c]= list(zip(top.index, top.values))

    
for x in list(top_dict)[0:100]:
    print("key {}, value {} ".format(x,  top_dict[x]))
    

In [ ]:
# Look at the most common top words --> add them to the stop word list
from collections import Counter

# Let's first pull out the top 100 words for each comedian
words = []
for president in data:
    top = [word for (word, count) in top_dict[president]]
    for t in top:
        words.append(t)

print(words[:10])

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import imageio
import matplotlib.pyplot as plt
import nltk

# Image used in which our world cloud output will be
img1 = imageio.imread("Manifesto/itula.jpeg")
hcmask1 = img1

# Get 100 words based on the 
words_except_stop_dist = nltk.FreqDist(w for w in words[:100]) 
wordcloud = WordCloud(stopwords=set(STOPWORDS),background_color='black',mask=hcmask1).generate(" ".join(words_except_stop_dist))
plt.imshow(wordcloud,interpolation = 'bilinear')
fig=plt.gcf()
fig.set_size_inches(10,12)
plt.axis('off')
plt.title("Top most common 100 words from Dr. Itula's Manifesto 2019",fontsize=20)
plt.tight_layout(pad=0)
plt.savefig('Manifesto_top_100.jpeg')

### **7. Sentiment Analysis of the Manifesto**

This is a set of Natural Language Processing (NLP) technique of analysing, identifying and categorizing opinions expressed in a piece of text, in order to determine whether the writer's attitude towards a particular topic, product, politics, services, brands etc. is positive, negative, or neutral. This data holds immense value in the fields of marketing analysis, public relations, product reviews, net promoter scoring, product feedback, and customer service, for example.

In [ ]:
from collections import defaultdict
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

Lets decide  which model we should use, between **TextBlob** and **VADER** for analysis of our text. We will therefore use TextBlob for its simplcity, and since VADER is specifically for analysis of social media data.  

#### **7.1 TextBlob function - returns two properties**

**Polarity:** a float value which ranges from [-1.0 to 1.0] where 0 indicates neutral, +1 indicates most positive statement and -1 rindicates  most negative statement.

**Subjectivity:** a float value which ranges from [0.0 to 1.0] where 0.0 is most objective while 1.0 is most subjective. Subjective sentence expresses some personal opinios, views, beliefs, emotions, allegations, desires, beliefs, suspicions, and speculations where as objective refers to factual information.

In [ ]:
blob = TextBlob(clean_data)
blob.sentiment

We can see that the polarity is **0.07** which means that the document is **neutral** and **0.47** subjectivity refers almost factual information in the document rather than public opinions, beliefs and so forth.

## **8. Text Summarization of the Document**

![title](Manifesto/text_summary.png)


Text summarization is an important NLP task which is a way of producing a concise and fluent summary of a perticular textin an article, journal, book, comment review, etc while also preseving the key information and overall meaning. Its is divided into categories, i.e., **extraction** and **abstraction**. Extractive methods select a subset of existing words, phrases, or sentences in the original text to form a summary. In contrast, abstractive methods first build an internal semantic representation and then use natural language generation techniques to create a summary.


Build a similarity matrix $\rightarrow$  generate rank based on matrix  $\rightarrow$ pick top N sentences for summary.

In [ ]:
# Importing all the necessary libraries
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Create a vectors between two sntences and calculate the cosine angel between them.

In [ ]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
# One out of 5 words differ => 0.8 similarity
print(sentence_similarity("This is a good sentence".split(), "This is a bad sentence".split()))
 
# One out of 2 non-stop words differ => 0.5 similarity
print(sentence_similarity("This is a good sentence".split(), "This is a bad sentence".split(), stopwords.words('english')))
 
# 0 out of 2 non-stop words differ => 1 similarity (identical sentences)
print(sentence_similarity("This is a good sentence".split(), "This is a good sentence".split(), stopwords.words('english')))
 
# Completely different sentences=> 0.0
print(sentence_similarity("This is a good sentence".split(), "I want to go to the market".split(), stopwords.words('english')))


We will use **unsupervised machine learning** approach to find the sentences similarity and rank them using the **cosine similarity** approach. Cosine simirality measures and calculates the similarity between two non-zero vectors of an inner product space by measuring the cosine angle between them.  One benefit of the **cosine similarity** approach is, there;s no need to train and build a model prior start using it for your project.

In [ ]:
#print(sentences)

# get the english list of stopwords
#stop_words = stopwords.words('english')
 
def build_similarity_matrix(lower_case, stopwords=None):
    # Create an empty similarity matrix
    S = np.zeros([len(lower_case), len(lower_case)])
 
 
    for idx1 in range(len(lower_case)):
        for idx2 in range(len(lower_case)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(lower_case[idx1], lower_case[idx2], stop_words)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
 
    return S

In [ ]:
#len(sentences)
#S = build_similarity_matrix(sentences, stop_words)    
#S

Lets define a function to genrate our summary of the whole document. Also note we will be calling other helper function to keep our summarization pipeline going.

In [ ]:
def generate_summary(lower_case, top_n=5):
    # Remove all the stopwords in the document
    stop_words = stopwords.words('english')
    summarize_text = []
    
    
    
    #Read text and tokenize
    #lower_case  = nltk.word_tokenize(lower_case)
    
   
    
    #Generate similarity matrix across sentences
    sentence_similarity  = build_similarity_matrix((lower_case, stop_words))
    
    #Rank sentences in similarity matrix
    sentence_similiraty_graph = nx.from_numpy_array(sentence_similarity)
    scores = nx.pagerank(sentence_similiraty_graph)
    
    
    #Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(lower_case)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence) 
    
    for i in range(top_n):
        summarize_text.append(' '.join(ranked_sentence[i][1]))
        
    #Output the summarized text
    print('Summarized Text: \n', '. '.join(summarize_text))
       

In [ ]:
#generate_summary(lower_case, 2)

Been trying to implement the tetxrank algorithm, however we keep getting an error whenever we call the `generate_summary()` function. Feel free to clarify on the error in order to generate a summary using this algorithm. With that said we found a simplest way to do it using just one line of code as shown in the following section.

## **8.1 Gensim Package**


Lets summarize our text sing the `gensim` package which is a  module for topic modelling for humans.  Summarizing is based on ranks of text sentences using a variation of the TextRank algorithm. see online documentation [gensim](https://radimrehurek.com/gensim/summarization/summariser.html) 

In [ ]:
# imports
from gensim.summarization.summarizer import summarize

In [ ]:
# Print out our summarized text of the document which was converted to lower case, remember we could have opted to remove stopwords as well.

print(summarize(lower_case))



This was our summarized text from the document. Note: there are various summarization packages like `TextTeaser`, `PyTeaser` , etc which one can use to accomplish this with only one line of code. 

## **9. Topic modeling using LDA** 

Topic modeling can be seen as a type of statistical modeling for discovering the abstract 'topics' that are presented in a myriad of documents (it can be a single document). A topic  is considered as a collection of prevalent keywords that are typical representatives. Its through keywords in which one determines what the topic is all about.

### What is LDA?
$\rightarrow$ Latent Dirichlet Allocation(LDA) is a popular algorithm for topic modeling with excellent implementations in the Python’s `gensim`package. We will therefore use LDA  to classify text in our document to a particular topic. I works by building a topic per document model and words per topic model, from Dirichlet distribution models in statistics.

In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
#import graphlab as gl
#import pyLDAvis.graphlab
import pyLDAvis.gensim  # don't skip this

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

Lets create a dictionary and corpus needed for topic modeling which are the two crucial inputs in implementint the LDA topc model

In [ ]:
data  = []
data.append(clean_text(lower_case))

In [ ]:
# This time we use spacy for lemmatizarion 
import spacy

# Second lemmatization of our data
def lemmatization(data, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_output = []
    for sent in data:
        doc = nlp(" ".join(sent)) 
        texts_output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_output

nlp = spacy.load('en', disable=['parser', 'ner'])

# Lemmatize keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


# View
print(corpus[:1])

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of word id and word frequency in the document, i.e. [0,1] word_id 0 means the word id appers first in the document and word frequency it appears once in the document. For Human readable formart see `cell [87-89]` or else run the following code `[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]`.  

### 9.1 Building the topic model with LDA


Apart from that, `alpha` and `beta` are hyperparameters that affect sparsity of the topics. Both with default values to 1.0/num_topics prior.

`alpha` is the per-document topic distribution, in simple terms it is a matrix where each row is a document and each column is a topic.
`beta` is the per-topic word distribution, also in simple terms it is a matrix where each row represents a topic and each column represents a word. 


Also see online documentation for `gensim.gensim.models.ldamodel.LdaModel()` parameters

In [ ]:
# LDA model

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, alpha='auto', num_topics=20, random_state=100,
                                           update_every=1, passes=20, per_word_topics=True)


In [ ]:
# Lets view the topics in our model
print(lda_model.print_topics())
doc_lda  = lda_model[corpus]

How do we make sense of this?

For topic `0` which is intrepreted as `'0.001*"need" + 0.001*"people" + 0.001*"namibian" + 0.001*"government" + 0.001*"citizen" + 0.001*"youth" + 0.001*"benefit" + 0.001*"system" + 0.001*"provide" + 0.001*"make"'`

**What does it mean?**  it means that the top 10 keywords that are part of this topic are: `need`, `people`, `namibian`, `government`, `citizen`, `youth`, `benefit`, `system`, `provide` and `make`. The numbers before the words represent the weight of the specific word on that topic, example `need` in topic `0` weigh `0.001` and this is much for all the words top 10 words in topic `0`. 


Can we deduced what topic this could be by looking on the top 10 keyswords? like what topic could trigger one to talk about `need`, `people`, and so on? We can perhaps summarize it to **POLITICS - Namibian People**

This can be done for all the remaining topics to see wether we can come up with a close judgement of each topic.


![titile](Manifesto/topic_model.png)

### 9.2 Model Perplexity and Coherence Score


Lets evaluate our model by computing the perplexity and topic coherence. Before that we need to understang what model perplexity and coherence is.
**Perplexity** is an evaluation  metric on how probable (predictive likelihood) new unseen data is given the model that was learned earlier. **Topic coherence** measures a score on a single topic through measuring the degree of semantic similarity of all the high scoring words in a topic. Both model perplexity and topic score provide a convenient measure to judge how good a given topic model is.  

In [ ]:
# Print model perplexity
print('\nPerplexity:', lda_model.log_perplexity(corpus))


# Coherence Score

coherence_model_lda = CoherenceModel(lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\Coherence Score:', coherence_lda)

### 9.3 Visualize topic's keywords

In [ ]:
pyLDAvis.enable_notebook()
vis_topics = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [ ]:
vis_topics

What can we deduce from graph output?

On the left it shows a buble in which it represnt a topic, also note we are supposed to have 20 bubles since we chose to have 20 topics in our model earlier on. The larger the buble, the more dominant that topic is. 


To determine a good topic, the bubble on the left would be big and non-overlapping throught the chart and being in spread along all the quadrants in stead of being clustered. Also note we only have one big bubble which means that we classified our topic 1 very well, however if you look closely you will see a black dot which seem to have all the remaining topic clustered together and overlaping each other in one place.



If you move the cursor on the bubble, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic, i.e, top 30 most relevant words. Clicking the  `Previous Topic` and `Next Topic`  buttons on the upper left, we again see the words on the right-hand side updating. However, there isn't much significant changes in most of the words in all the topics as they are all dominated by words like `need`, `youth`, `people`, `government`, `country` , `namibian` and so on... We can say 

With the output, we can say that the topics in our document exacerbate around **politics**, **Namibian politics** to be specific. Thus we have successfully build a good looking topic model.

### **10. Conclusion**


* We have succefully analysed the document eventhough there are many models that can be use to throughly get an idea of the whole document.

* The data cleaning process had issues in which there we characters that were not english and thus have tempered with our analysis, this can be improved.

* Getting the top 100 most common words resonate well with me on a personal level looking at the state in which Namibia is and what that is needed to be done. This could be because of the word **youth** since I happen fall under that group, haha :-D

* For sentiment analysis, we found the document to be neutral with almost factual information in the document rather than public opinions, beliefs and so forth.

* The summarization of the document was quite challenging using the TextRank algorimth together with the implementaion of the similarity matrix, however it was easily done with the `gensim` package in one line of code, CAN YOU IMAGINE? Me neither :-;

* Topic modelling  was successful, in which we could say that the document is focused on Namibian politics. 

**Hope you enjoyed reading this. I would appreciate if you leave your thoughts in the comments section. And dont forget to upvote if you liked it**